In [1]:
%xmode Minimal

Exception reporting mode: Minimal


In [2]:
%%capture pipoutput
%pip install python-dotenv boto3 pandas pyarrow fastparquet 

In [3]:
%%capture pipoutput
%pip install --upgrade osc-ingest-tools

In [4]:
import osc_ingest_trino as osc
osc.load_credentials_dotenv()

engine = osc.attach_trino_engine(verbose=True)

using connect string: trino://erikerlandson@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443


In [5]:
# bucket must be configured with credentials for trino, and accessible to the hive catalog
# You may need to use a different prefix here depending on how you name your credentials.env variables
bucket = osc.attach_s3_bucket('S3_OSCCL2')

In [6]:
import os
# create demo table named per user
# avoids problems with users reusing table names and associated permission problems
uniq = os.environ['TRINO_USER']
uniq

'erikerlandson'

In [7]:
ingest_catalog = 'osc_datacommons_hive_ingest'
ingest_schema = 'ingest'

In [8]:
partition_columns = ['year']

In [9]:
iceberg_catalog = 'osc_datacommons_dev'
iceberg_schema = 'sandbox'
iceberg_table = f'iceberg_demo_{uniq}'

In [10]:
import random
import pandas as pd

datasize = 1000
yeardata = [random.randrange(2020,2023) for x in range(datasize)]
metricdata = [random.random() for x in range(datasize)]

df1 = pd.DataFrame(data={"year": yeardata, "metric": metricdata})
df1.convert_dtypes()

,year,metric
0,2020,0.660268
1,2020,0.455403
2,2022,0.134238
3,2022,0.23197
4,2020,0.685925
...,...,...
995,2020,0.565553
996,2022,0.356613
997,2021,0.608207
998,2021,0.036636


In [11]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1000 non-null   int64  
 1   metric  1000 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 15.8 KB


In [12]:
osc.fast_pandas_ingest_via_hive(
    df1,
    engine,
    iceberg_catalog, iceberg_schema, iceberg_table,
    bucket, ingest_catalog, ingest_schema,
    partition_columns = partition_columns,
    overwrite = True,
    verbose = True
)

enforcing dataframe partition column order

verifying existence of table osc_datacommons_dev.sandbox.iceberg_demo_erikerlandson
create table if not exists osc_datacommons_dev.sandbox.iceberg_demo_erikerlandson (
    metric double,
    year bigint
) with (
    format = 'parquet',
    partitioning = array['year']
)
[(True,)]

staging dataframe parquet to s3 osc-datacommons-s3-bucket-dev02
/tmp/ingest_temp_790cfbe8/year=2020/b751a7265ec3482f81312d77c101c1c5-0.parquet  -->  trino/ingest/ingest_temp_790cfbe8/year=2020/b751a7265ec3482f81312d77c101c1c5-0.parquet
/tmp/ingest_temp_790cfbe8/year=2021/b751a7265ec3482f81312d77c101c1c5-0.parquet  -->  trino/ingest/ingest_temp_790cfbe8/year=2021/b751a7265ec3482f81312d77c101c1c5-0.parquet
/tmp/ingest_temp_790cfbe8/year=2022/b751a7265ec3482f81312d77c101c1c5-0.parquet  -->  trino/ingest/ingest_temp_790cfbe8/year=2022/b751a7265ec3482f81312d77c101c1c5-0.parquet

declaring intermediate hive table osc_datacommons_hive_ingest.ingest.ingest_temp_790cfbe8
cre

In [13]:
sql = f"""
select * from {iceberg_catalog}.{iceberg_schema}.{iceberg_table}
"""
df = pd.read_sql(sql, engine).convert_dtypes()
df

,metric,year
0,0.355128,2021
1,0.281124,2021
2,0.971087,2021
3,0.880857,2021
4,0.886227,2021
...,...,...
995,0.136344,2022
996,0.11782,2022
997,0.400703,2022
998,0.61321,2022
